<a href="https://colab.research.google.com/github/mihir238/mywork/blob/main/GAN_practice2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import keras
import matplotlib.pyplot as plt


In [ ]:
from numpy import zeros, ones, expand_dims, asarray
from numpy.random import randn, randint
from keras.datasets import fashion_mnist
#from keras.optimizers import Adam
from keras.models import Model, load_model
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import Conv2D, Conv2DTranspose, Concatenate
from keras.layers import LeakyReLU, Dropout, Embedding
from keras.layers import BatchNormalization, Activation
from keras import initializers
from keras.initializers import RandomNormal
#from keras.optimizers import Adam, RMSprop, SGD
from matplotlib import pyplot
from math import sqrt

In [ ]:
from tensorflow import keras

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam, RMSprop, SGD

In [ ]:
from keras.models import Model, Sequential


In [ ]:
from tensorflow.keras.datasets import mnist

In [ ]:
class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build and compile the generator
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

        # The generator takes noise as input and generated imgs
        z = Input(shape=(100,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        noise_shape = (100,)

        model = Sequential()

        model.add(Dense(256, input_shape=noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        img_shape = (self.img_rows, self.img_cols, self.channels)

        model = Sequential()

        model.add(Flatten(input_shape=img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()
        #(X_train, _), (_, _) = fashion_mnist.load_data()
        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (half_batch, 100))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, 100))

            # The generator wants the discriminator to label the generated samples
            # as valid (ones)
            valid_y = np.array([1] * batch_size)

            # Train the generator
            g_loss = self.combined.train_on_batch(noise, valid_y)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, 100))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        #fig.savefig("gan/images/mnist_%d.png" % epoch)
        #fig.savefig("https://drive.google.com/drive/u/0/folders/1MKfN67zjZWZ9A-AIEpuqr9IhUjrqIYGW/mnist_%d.png" % epoch)
        #fig.savefig("C:/Users/mihir/Desktop/Untitled Folder/GAN/New folder/mnist_%d.png" % epoch)
        
        plt.close()


if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=30000, batch_size=32, save_interval=200)

Streaming output truncated to the last 5000 lines.
25000 [D loss: 0.688116, acc.: 62.50%] [G loss: 0.782100]
25001 [D loss: 0.734065, acc.: 43.75%] [G loss: 0.821896]
25002 [D loss: 0.714553, acc.: 53.12%] [G loss: 0.804235]
25003 [D loss: 0.673817, acc.: 62.50%] [G loss: 0.870000]
25004 [D loss: 0.613789, acc.: 71.88%] [G loss: 0.807166]
25005 [D loss: 0.726681, acc.: 53.12%] [G loss: 0.812569]
25006 [D loss: 0.678690, acc.: 53.12%] [G loss: 0.909140]
25007 [D loss: 0.637753, acc.: 56.25%] [G loss: 0.895369]
25008 [D loss: 0.635089, acc.: 65.62%] [G loss: 0.903900]
25009 [D loss: 0.690822, acc.: 46.88%] [G loss: 0.979449]
25010 [D loss: 0.664489, acc.: 53.12%] [G loss: 0.819565]
25011 [D loss: 0.729199, acc.: 53.12%] [G loss: 0.801016]
25012 [D loss: 0.732144, acc.: 53.12%] [G loss: 0.830486]
25013 [D loss: 0.676588, acc.: 56.25%] [G loss: 0.840051]
25014 [D loss: 0.672875, acc.: 53.12%] [G loss: 0.783097]
25015 [D loss: 0.605723, acc.: 65.62%] [G loss: 0.820086]
25016 [D loss: 0.7124